In [1]:
#%reset
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
#%matplotlib widget

import cmocean
import glob

import os

import math
from datetime import timedelta, datetime
from operator import attrgetter

from datetime import timedelta as delta

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import pop_tools
from scipy.optimize import curve_fit


In [2]:
grid = pop_tools.get_grid('POP_gx1v7')[['TAREA', 'KMT', 'TLAT', 'TLONG', 'REGION_MASK']]

In [3]:
def process(da):
    
    da['ALK_excess'] = da.ALK - da.ALK_ALT_CO2
    da['DIC_excess'] = da.DIC - da.DIC_ALT_CO2
    da['pCO2_SURF_excess'] = da.pCO2SURF - da.pCO2SURF_ALT_CO2
    
    return da

Read data

In [4]:
%%time
#fpath = '/glade/scratch/mclong/OAE-Global-Efficiency/archive/smyle-fosi.North_Atlantic_basin_yearlong.alk-forcing-North_Atlantic_basin_yearlong.063-01/ocn/hist/'

location  = 'North_Atlantic_basin' #North_Atlantic_basin or North_Pacific_basin
polygon = '000'                    #for Atlantic: 0, 16, 36, 63; for Pacific: 105, 155 added 51, 64, 184, 185

# 1 year continuous release:
fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Elizabeth_IRF_Experiments/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1year/ocn/hist/'
da_1yr = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1year.pop.h.*.nc')
print(da_1yr)

# 5 year continuous release:
fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Elizabeth_IRF_Experiments/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-5year/ocn/hist/'
da_5yr = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-5year.pop.h.*.nc')
print(da_5yr)


<xarray.Dataset>
Dimensions:                     (d2: 2, nlat: 384, nlon: 320, time: 240, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                        (time) object 0347-02-01 00:00:00 ... 0367-01...
  * z_t                         (z_t) float32 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                    (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                         (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                     (z_w_top) float32 0.0 1e+03 ... 5e+05 5.25e+05
  * z_w_bot                     (z_w_bot) float32 1e+03 2e+03 ... 5.5e+05
    ULONG                       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                        (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT                        (nlat, nlon) float64 dask.array

In [5]:
#Month-long release for each month:
fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Mengyang_Global_OAE_Experiments/archive/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-01/ocn/hist/'
da_01 = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-01.pop.h.*.nc')
print(da_01)

fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Mengyang_Global_OAE_Experiments/archive/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-04/ocn/hist/'
da_04 = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-04.pop.h.*.nc')
print(da_04)

fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Mengyang_Global_OAE_Experiments/archive/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-07/ocn/hist/'
da_07 = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-07.pop.h.*.nc')
print(da_07)

#Month-long release for each month:
fpath = f'/glade/campaign/cesm/development/bgcwg/projects/OAE-Global-Efficiency/Mengyang_Global_OAE_Experiments/archive/smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-10/ocn/hist/'
da_10 = xr.open_mfdataset(fpath + f'smyle-fosi.{location}.alk-forcing-{location}.{polygon}-1999-10.pop.h.*.nc')
print(da_10)

<xarray.Dataset>
Dimensions:                     (d2: 2, nlat: 384, nlon: 320, time: 180, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                        (time) object 0347-02-01 00:00:00 ... 0362-01...
  * z_t                         (z_t) float32 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                    (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                         (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                     (z_w_top) float32 0.0 1e+03 ... 5e+05 5.25e+05
  * z_w_bot                     (z_w_bot) float32 1e+03 2e+03 ... 5.5e+05
    ULONG                       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                        (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                       (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT                        (nlat, nlon) float64 dask.array

In [6]:
print('done')

done


In [7]:
da_1yr = process(da_1yr)
da_5yr = process(da_5yr)

da_01 = process(da_01)
da_04 = process(da_04)
da_07 = process(da_07)
da_10 = process(da_10)

In [8]:
def excess_dic(da):
    
    dic_inv = ((da.DIC - da.DIC_ALT_CO2) * da.dz * da.TAREA).sum(['z_t', 'nlat', 'nlon']) * 1e-9
    dic_inv = dic_inv.compute()
    
    return dic_inv

In [9]:
#Still need to multiply this by the number of days in total (since we're summing over months that have unequal day numbers)
def cal_added_ALk(da):
    '''Total added ALK, in mol'''
    #ALk_added_total = (da.ALK_FLUX * da.TAREA).sum(['time','nlat', 'nlon'])*1e-9*86400 # mol/day for that one month

    ALk_added_total = (da.ALK_FLUX * da.TAREA).sum(['nlat', 'nlon']).cumsum(['time'])*1e-9*86400 # mol/day for that one month
    return ALk_added_total.values

In [ ]:
total_exc_alk_1yr = (cal_added_ALk(da_1yr)*365/12)
total_exc_alk_5yr = (cal_added_ALk(da_5yr)*365/12)
total_exc_alk_01  = (cal_added_ALk(da_01)*31)
total_exc_alk_04  = (cal_added_ALk(da_04)*30)
total_exc_alk_07  = (cal_added_ALk(da_07)*31)
total_exc_alk_10  = (cal_added_ALk(da_10)*31)

exc_dic_1yr = excess_dic(da_1yr)
exc_dic_5yr = excess_dic(da_5yr)
exc_dic_01  = excess_dic(da_01)
exc_dic_04  = excess_dic(da_04)
exc_dic_07  = excess_dic(da_07)
exc_dic_10  = excess_dic(da_10)

ALk_added_1yr = (da_1yr.ALK_FLUX * da_1yr.TAREA).sum(['nlat', 'nlon'])*1e-9*86400  # mol/day
ALk_added_5yr = (da_5yr.ALK_FLUX * da_5yr.TAREA).sum(['nlat', 'nlon'])*1e-9*86400  # mol/day 
ALk_added_01 =  (da_01.ALK_FLUX  * da_01.TAREA).sum(['nlat', 'nlon'])*1e-9*86400   
ALk_added_04 =  (da_04.ALK_FLUX  * da_04.TAREA).sum(['nlat', 'nlon'])*1e-9*86400  
ALk_added_07 =  (da_07.ALK_FLUX  * da_07.TAREA).sum(['nlat', 'nlon'])*1e-9*86400 
ALk_added_10 =  (da_10.ALK_FLUX  * da_10.TAREA).sum(['nlat', 'nlon'])*1e-9*86400  

print((exc_dic_1yr/total_exc_alk_1yr)[-1])
print((exc_dic_5yr/total_exc_alk_5yr)[-1])

In [ ]:
fig = plt.figure(figsize=(12,3))
ax = fig.add_subplot(131)
ax.plot(ALk_added_01.time, ALk_added_01.values, c='#BFBFFF', label='1m Jan.')
ax.plot(ALk_added_04.time, ALk_added_04.values, c='#A3A3FF', label='1m Apr.')
ax.plot(ALk_added_07.time, ALk_added_07.values, c='#7879FF', label='1m Jul.')
ax.plot(ALk_added_10.time, ALk_added_10.values, c='#4949FF', label='1m Oct.')

ax.plot(ALk_added_1yr.time, ALk_added_1yr.values, label='1 year', c = 'gold')
ax.plot(ALk_added_5yr.time, ALk_added_5yr.values, label='5 year', c = 'orangered')

ax.set_title('ALK Flux')
ax.set_ylabel('ALK Flux (mol/day)')
ax.set_xlabel('Time (years)')

ax1 = fig.add_subplot(132)
ax1.plot(exc_dic_01.time, exc_dic_01, c='#BFBFFF', label='1m Jan.')
ax1.plot(exc_dic_04.time, exc_dic_04, c='#A3A3FF', label='1m Apr.')
ax1.plot(exc_dic_07.time, exc_dic_07, c='#7879FF', label='1m Jul.')
ax1.plot(exc_dic_10.time, exc_dic_10, c='#4949FF', label='1m Oct.')

ax1.plot(exc_dic_1yr.time, exc_dic_1yr, c = 'gold', label='1 year')
ax1.plot(exc_dic_5yr.time, exc_dic_5yr, c = 'orangered', label='5 year')

ax1.set_ylabel('DIC Inventory (mol)')
ax1.set_xlabel('Time (years)')
ax1.set_title('DIC Inventory')

ax2 = fig.add_subplot(133)
ax2.plot(exc_dic_01.time, exc_dic_01/total_exc_alk_01, c='#BFBFFF', label='1m Jan.')
ax2.plot(exc_dic_04.time, exc_dic_04/total_exc_alk_04, c='#A3A3FF', label='1m Apr.')
ax2.plot(exc_dic_07.time, exc_dic_07/total_exc_alk_07, c='#7879FF', label='1m Jul.')
ax2.plot(exc_dic_10.time, exc_dic_10/total_exc_alk_10, c='#4949FF', label='1m Oct.')

ax2.plot(exc_dic_1yr.time, exc_dic_1yr/total_exc_alk_1yr, c = 'gold', label='1 year')
ax2.plot(exc_dic_5yr.time, exc_dic_5yr/total_exc_alk_5yr, c = 'orangered', label='5 year')
#ax2.plot(exc_dic_5yr.time, exc_dic_5yr/total_exc_alk_5yr[-1], c = 'k', label='5 year')

ax2.set_ylabel('$\eta(t)$')
ax2.set_xlabel('Time (years)')
ax2.set_title('OAE Efficiency')


ax.set_xlim(da_5yr.time.values[0] - timedelta(days=30), da_5yr.time.values[-1] + timedelta(days=30))
ax1.set_xlim(da_5yr.time.values[0] - timedelta(days=30), da_5yr.time.values[-1] + timedelta(days=30))
ax2.set_xlim(da_5yr.time.values[0] - timedelta(days=30), da_5yr.time.values[-1] + timedelta(days=30))

ax.legend();  ax.grid();
#ax1.legend(); 
ax1.grid();
#ax2.legend(); 
ax2.grid();

plt.subplots_adjust(wspace=0.3)


# plt.savefig('/glade/work/mengyangz/GVP/figures/check_000_ALK_DIC_eff.png', 
#             dpi=200, bbox_inches='tight')

In [ ]:
def plot(da, tstep):
    
        fig = plt.figure(figsize=(10,9),dpi=150)

        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=-120))
        ax.stock_img()
        ax.coastlines(linewidth=0.5, color='blue', resolution='110m')


        sca = ax.pcolormesh(grid.TLONG,
                      grid.TLAT,
                      da.isel(time=tstep, z_t=0).ALK_excess, vmin=0,vmax=5,
                      transform=ccrs.PlateCarree(),
                      cmap=cmocean.cm.dense);  
        cb = fig.colorbar(sca, ax=ax, shrink=0.74)

        cb.ax.set_title('meq/m$^2$') 


        ax.set_extent([-130, 30, -50, 80], ccrs.PlateCarree(central_longitude=-120)) #originally -110 limit
        ax.set_xticks(np.arange(-130, 30, 20), crs=ccrs.PlateCarree(central_longitude=-120))
        ax.set_yticks(np.arange(-50, 80, 10), crs=ccrs.PlateCarree(central_longitude=-120))
        lon_formatter = LongitudeFormatter(zero_direction_label=False)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)  

        ax.stock_img()
        ax.add_feature(cfeature.LAND, facecolor='lightgray')

        ax.set_title('excess_ALK at surface ' + str(da.time.values[tstep])[:10], loc='left')
        #ax.set_title('DIC Surface Gas Flux, Alternative CO2' + str(da.time.values[tstep])[:10], loc='left')
        ax.grid()
    #     plt.close()

    #     fig.savefig(f'/glade/work/mengyangz/GVP/figures/figs2movies/Atlantic000/{tstep:04d}.png', dpi=300, bbox_inches='tight')

In [ ]:
.62/.78

In [ ]:
plot(da_01, 0) #vmax = 0.005

In [ ]:
def exponential_tau(t, eta_max, ktau, gtau, htau):
    '''
    t: time
    eta_max: initial condition (this doubles as intrinsic "max" efficiency, ~0.81)
    ktau: e-folding time of gas exchange
    gtau: e-folding time of loss to the deep
    htau: e-folding time of gas exchange for the deep reservoir
    '''
    alpha = gtau/(ktau+gtau)
    tau_kg = 1.0/(1/ktau+1/gtau)
    return eta_max * (1 -  alpha*np.exp(-(1/ktau+1/gtau)*t)
                         - (1-alpha)*np.exp(-t/htau)
                         - (1-alpha)*(tau_kg/(htau - tau_kg))*(np.exp(-t/htau) - np.exp(-t/tau_kg))
                    )


def fit_params_tau(x_data, y_data, bounds=None, initial=None, pink=None, ping=None, pinh=None):
    '''Return fitted parameters'''
    # Perform the curve fitting
    if bounds is None:
        #bounds=[[ 0.849,  1.96,   0,       0],
        #        [ 0.851,  1.98,  2000,  10000]]
        bounds=[[ 0.80,  2,    0,       5],
                [ 0.88, 50 ,  2000,  10000]]
    if initial is None:
       initial = [(bounds[0][0]+bounds[1][0])/2,
                   np.clip(4,bounds[0][1],bounds[1][1]),
                   np.clip(25,bounds[0][2],bounds[1][2]),
                   np.clip(70,bounds[0][3],bounds[1][3])]
    if pink is not None:
      bounds[0][1] = pink
      bounds[1][1] = pink+0.01
      initial[1] = pink
    if ping is not None:
      bounds[0][2] = ping
      bounds[1][2] = ping+0.01
      initial[2] = ping
    if pinh is not None:
      bounds[0][3] = pinh
      bounds[1][3] = pinh+0.01
      initial[3] = pinh

    popt, pcov = curve_fit(exponential_tau, x_data, y_data, p0=initial,
                           method='trf', maxfev=10000, bounds=bounds)

    stdev = np.sqrt(np.diag(pcov))
    #print("%.2f %.1f %.1f %.1f"%tuple(stdev))
    return popt

In [ ]:
def fit_to_curve_tau(x_data, y_data, s, show=False, onlydata=False, *args, **kwargs):
 
  if  len(y_data)<=1: # but abort if there's no data
    return np.nan,np.nan,np.nan,np.nan

  eta_max, ktau, gtau, htau = fit_params_tau(x_data, y_data, *args, **kwargs)
  if show:
    if onlydata:
      plt.plot(x_data, y_data)
    else:
      plt.plot(x_data, y_data, c="black",ls="dashed", label=None)
      plt.plot(x_data, exponential_tau(x_data, eta_max, ktau, gtau, htau), c = ["#BFBFFF","#A3A3FF","#7879FF","#4949FF"][s],
             label = r"%s: eta=%.2f t$_{gas}$=%.1fm t$_{loss}$=%.1fm,  t$_{slow}$=%.1fm"%(["Jan","Apr","Jul","Oct"][s],eta_max, ktau, gtau, htau))

    plt.legend(loc='lower right')
  return eta_max, ktau, gtau, htau

In [ ]:

x_data = np.array(range(15*12))  # time in months

y_data_01 = (exc_dic_01/total_exc_alk_01).values; y_data_01 = y_data_01[~np.isnan(y_data_01)]  # get rid of nans
y_data_04 = (exc_dic_04/total_exc_alk_04).values; y_data_04 = y_data_04[~np.isnan(y_data_04)]  # get rid of nans
y_data_07 = (exc_dic_07/total_exc_alk_07).values; y_data_07 = y_data_07[~np.isnan(y_data_07)]  # get rid of nans
y_data_10 = (exc_dic_10/total_exc_alk_10).values; y_data_10 = y_data_10[~np.isnan(y_data_10)]  # get rid of nans

[eta_max_01, ktau_01, gtau_01, htau_01] = fit_to_curve_tau(x_data,y_data_01,0,show=True)
[eta_max_04, ktau_04, gtau_04, htau_04] = fit_to_curve_tau(x_data,y_data_04,1,show=True)
[eta_max_07, ktau_07, gtau_07, htau_07] = fit_to_curve_tau(x_data,y_data_07,2,show=True)
[eta_max_10, ktau_10, gtau_10, htau_10] = fit_to_curve_tau(x_data,y_data_10,3,show=True)

#plt.plot(t,test1,'--r')
#plt.plot(t,test2,'--y')
#plt.plot(convolution/np.nansum(ALk_added_total),'y')
plt.ylabel('$\eta$')
plt.xlabel('Time in months')
plt.title('Month-long releases and fitting')
#plt.plot([0, 190],[0.85,0.85])
plt.grid('on')
plt.xlim([0,190])
#plt.ylim([.0,.9])

In [ ]:
t = np.array(range(20*12))

IRF_01 = exponential_tau(t, eta_max_01, ktau_01, gtau_01, htau_01)
IRF_04 = exponential_tau(t, eta_max_04, ktau_04, gtau_04, htau_04)
IRF_07 = exponential_tau(t, eta_max_07, ktau_07, gtau_07, htau_07)
IRF_10 = exponential_tau(t, eta_max_10, ktau_10, gtau_10, htau_10)

IRF_02 = IRF_01*(2/3) + IRF_04*(1/3)
IRF_03 = IRF_01*(1/3) + IRF_04*(2/3)
IRF_05 = IRF_04*(2/3) + IRF_07*(1/3)
IRF_06 = IRF_04*(1/3) + IRF_07*(2/3)
IRF_08 = IRF_07*(2/3) + IRF_10*(1/3)
IRF_09 = IRF_07*(1/3) + IRF_10*(2/3)
IRF_11 = IRF_10*(2/3) + IRF_01*(1/3)
IRF_12 = IRF_10*(1/3) + IRF_01*(2/3)

IRF_13 = IRF_01; IRF_25 = IRF_01; IRF_37 = IRF_01; IRF_49 = IRF_01;
IRF_14 = IRF_02; IRF_26 = IRF_02; IRF_38 = IRF_02; IRF_50 = IRF_02;
IRF_15 = IRF_03; IRF_27 = IRF_03; IRF_39 = IRF_03; IRF_51 = IRF_03;
IRF_16 = IRF_04; IRF_28 = IRF_04; IRF_40 = IRF_04; IRF_52 = IRF_04;
IRF_17 = IRF_05; IRF_29 = IRF_05; IRF_41 = IRF_05; IRF_53 = IRF_05;
IRF_18 = IRF_06; IRF_30 = IRF_06; IRF_42 = IRF_06; IRF_54 = IRF_06;
IRF_19 = IRF_07; IRF_31 = IRF_07; IRF_43 = IRF_07; IRF_55 = IRF_07;
IRF_20 = IRF_08; IRF_32 = IRF_08; IRF_44 = IRF_08; IRF_56 = IRF_08;
IRF_21 = IRF_09; IRF_33 = IRF_09; IRF_45 = IRF_09; IRF_57 = IRF_09;
IRF_22 = IRF_10; IRF_34 = IRF_10; IRF_46 = IRF_10; IRF_58 = IRF_10;
IRF_23 = IRF_11; IRF_35 = IRF_11; IRF_47 = IRF_11; IRF_59 = IRF_11;
IRF_24 = IRF_12; IRF_36 = IRF_12; IRF_48 = IRF_12; IRF_60 = IRF_12;


In [ ]:
fig = plt.figure(figsize=(7,4),dpi=400)

color = iter(plt.cm.cool(np.linspace(0, 1, 12)))

   

for i in range(1,13):
    print(i)
    c = next(color)
    #print(globals()[f'IRF_{i:02d}'])
    plt.plot(t/12,globals()[f'IRF_{i:02d}'],label=f'IRF Month {i:02d}',c=c,linewidth=1)
#plt.legend()
plt.ylabel('$\eta$')
plt.xlabel('Time (years)')
plt.title('Monthly IRFs -- CA')
#plt.plot([0, 190],[0.85,0.85])
plt.grid('on')
plt.xlim([0,2.5])
plt.ylim([0,.8])
plt.xticks(np.linspace(0,2.5,6))

#plt.plot(t,

In [ ]:
# for x in [1,4,7,10]:
#     globals()[f'alpha_{x:02d}'] = globals()[f'gtau_{x:02d}']/(globals()[f'ktau_{x:02d}']+globals()[f'gtau_{x:02d}']); 
#     tau_kg_01 = 1.0/(1/ktau_01+1/gtau_01)
#     globals()[f'IRF_{x:02d}']=
#     #print(var)
#     #my_data[f’Data_{i:2d}’] = i * 10 
# IRF_01

In [ ]:
ALk_added_total = ALk_added_1yr*365/12  # mol/month 
convolution_simple1 = np.convolve(ALk_added_total, IRF_01)

ALk_added_local = np.zeros(ALk_added_total.shape); ALk_added_local[0]=ALk_added_total[0]
convolution1    = np.zeros(ALk_added_total.shape);
for i in range(0,12):
    print(i)
    #ALk_added_local = np.zeros(ALk_added_total.shape); ALk_added_local[i]=ALk_added_total[i]
    convolution_local = np.convolve(ALk_added_local, globals()[f'IRF_{i+1:02d}'])
    convolution1[i:240] = convolution1[i:240] + convolution_local[0:240-i]
#convolution1.shape

In [ ]:
ALk_added_total = ALk_added_5yr*365/12  # mol/month 
convolution_simple5 = np.convolve(ALk_added_total, IRF_01)

ALk_added_local = np.zeros(ALk_added_total.shape); ALk_added_local[0]=ALk_added_total[0]
convolution5    = np.zeros(ALk_added_total.shape);
for i in range(0,60):
    #print(f'IRF_{i+1:02d}')
    #ALk_added_local = np.zeros(ALk_added_total.shape); ALk_added_local[i]=ALk_added_total[i]
    convolution_local = np.convolve(ALk_added_local, globals()[f'IRF_{i+1:02d}'])
    convolution5[i:240] = convolution5[i:240] + convolution_local[0:240-i]
#convolution1.shape

In [ ]:
x_data.shape

In [ ]:
#fig = plt.figure(figsize=(6,3),dpi=400)
fig = plt.figure(figsize=(10,6),dpi=400)

x_data = np.array(range(20*12))/12  # time in months
y_data = (exc_dic_1yr/total_exc_alk_1yr).values
y_data = y_data[~np.isnan(y_data)]  # get rid of nans

plt.plot(x_data[0:180], (exc_dic_01/total_exc_alk_01),c = 'k',label='Month-long release',linewidth=1)

plt.plot(x_data, y_data,c='coral',label='1 yr. release',linewidth=1.5)
plt.plot(x_data,convolution1/total_exc_alk_1yr,'--',c='coral',label='1 yr: seasonal IRF prediction',linewidth=1.5)
#plt.plot(x_data,convolution_simple1[0:240]/total_exc_alk_1yr,':',c='coral',label='1 yr: Jan. IRF prediction',linewidth=1.5)

y_data = (exc_dic_5yr/total_exc_alk_5yr).values
y_data = y_data[~np.isnan(y_data)]  # get rid of nans

plt.plot(x_data, y_data,c='darkcyan',label='5 yr. release',linewidth=1.5)
plt.plot(x_data,convolution5/total_exc_alk_5yr,'--',c='darkcyan',label='5 yr: seasonal IRF prediction',linewidth=1.5)
#plt.plot(x_data,convolution_simple5[0:240]/total_exc_alk_5yr,':',c='darkcyan',label='5 yr: Jan. IRF prediction',linewidth=1.5)


plt.ylabel('$\eta$')
plt.xlabel('Time (Years)')
plt.title('IRF Predictions and Model Result Comparison -- California')
#plt.plot([0, 190],[0.85,0.85]) #BFBFFF
plt.grid('on')
plt.xlim([0,20])
plt.ylim([.0,.9])
plt.xticks(np.linspace(0,20,21))
plt.legend(loc='upper left',fontsize=14)

In [ ]:
x_data = np.array(range(20*12))  # time in months
y_data = (exc_dic_1yr/total_exc_alk_1yr).values
y_data = y_data[~np.isnan(y_data)]  # get rid of nans

plt.plot(x_data, y_data/(convolution1/total_exc_alk_1yr),c='orangered',label='1 Year-long release/IRF prediction')
print((y_data/(convolution1/total_exc_alk_1yr))[-1])

y_data = (exc_dic_5yr/total_exc_alk_5yr).values
y_data = y_data[~np.isnan(y_data)]  # get rid of nans

plt.plot(x_data, y_data/(convolution5/total_exc_alk_5yr),c='blue',label='5 Year-long release/IRF prediction')
print((y_data/(convolution5/total_exc_alk_5yr))[-1])

#plt.plot(x_data, (exc_dic063/total_exc_alk063),'k--',label='Month-long release')

plt.ylabel('$\eta$ ratio')
plt.xlabel('Time in months')
plt.title('Ratios')
#plt.plot([0, 190],[0.85,0.85])
plt.grid('on')
plt.xlim([0,175])
plt.ylim([.8, 2])
plt.legend(loc='upper right')